In [1]:
!git clone https://github.com/mohamed-stifi/darija-llm-lab.git

fatal: destination path 'darija-llm-lab' already exists and is not an empty directory.


In [2]:
import os
os.chdir("darija-llm-lab")

In [3]:
%%capture
!python -m pip install --upgrade pip
!pip install uv
!uv pip install -r requirements.txt

In [4]:
#import os
#os.chdir("../")
!pwd

/kaggle/working/darija-llm-lab


In [5]:
from darija_llm_lab.components.data_ingestion.data_ingestion import DataIngestion
from darija_llm_lab.utils.common import ConfigurationManager

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-11 15:20:07.369691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754925607.710050     140 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754925607.808044     140 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


## Get configs

In [6]:
config_manager = ConfigurationManager(config_filepath="config/config.yaml")

In [7]:
# 1. Get configs
model_cfg = config_manager.get_model_config()
dapt_cfg = config_manager.get_dapt_config()
sft_config = config_manager.get_sft_config()
new_tokens = config_manager.new_tokens
wandb_config = config_manager.get_wandb_config()
optuna_config = config_manager.get_optuna_config()

In [8]:
sft_config

SFTConfig(input_model_path=PosixPath('artifacts/models/gemma-3n-E2B-it-darija-adapted'), output_adapters_path=PosixPath('artifacts/sft_adapters'), data=SFTDataConfig(dataset_id='MBZUAI-Paris/Darija-SFT-Mixture', test_size=0.1, seed=1234, debug_mode=True, debug_split_size=3000), peft=SFTPEFTConfig(r=16, lora_alpha=16, lora_dropout=0.05, bias='none', random_state=3407, target_modules=BoxList(['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'])), trainer=SFTTrainerConfig(output_dir='artifacts/sft_training_results', per_device_train_batch_size=10, gradient_accumulation_steps=10, eval_strategy='steps', eval_steps=10, save_strategy='steps', save_steps=30, warmup_steps=10, max_steps=100, learning_rate=2e-05, logging_steps=5, seed=3407, metric_for_best_model='eval_loss', load_best_model_at_end=True, greater_is_better=False, early_stopping_patience=3, optim='adamw_torch', weight_decay=0.01))

In [9]:
dapt_cfg

DAPTConfig(output_model_path=PosixPath('artifacts/models/gemma-3n-E2B-it-darija-adapted'), data=DAPTDataConfig(dataset_id='mohamed-stifi/darija-combined-dataset', text_column='text', debug_mode=True, debug_split_size=5000), trainer=DAPTTrainerConfig(output_dir='artifacts/dapt_training_results', per_device_train_batch_size=8, gradient_accumulation_steps=8, max_steps=200, learning_rate=5e-06, logging_steps=10, save_steps=50, warmup_steps=20, seed=3407, optim='adamw_torch', weight_decay=0.01))

In [10]:
dapt_cfg.data

DAPTDataConfig(dataset_id='mohamed-stifi/darija-combined-dataset', text_column='text', debug_mode=True, debug_split_size=5000)

## Load and prepare model

In [11]:
from darija_llm_lab.components.model_loader import ModelLoader

# 2. Load and prepare model
model_loader = ModelLoader(model_cfg)
model, tokenizer = model_loader.load_model_and_tokenizer()

--- Loading model from: unsloth/gemma-3n-E2B-it ---
==((====))==  Unsloth 2025.7.2: Fast Gemma3N patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

## load_dapt_dataset

In [12]:
data_ingestion = DataIngestion()

In [13]:
train_dataset = data_ingestion.load_dapt_dataset(dapt_cfg.data, tokenizer)


--- Loading and preparing DAPT dataset ---


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/4.51M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/35562 [00:00<?, ? examples/s]

In [14]:
train_dataset

Dataset({
    features: ['text', 'source'],
    num_rows: 5000
})

In [15]:
train_dataset['text'][0]

'من مور شي يومين خرجات سمية من دارها'

## load_sft_dataset

In [16]:
train_ds, eval_ds = data_ingestion.load_sft_dataset(sft_config.data, tokenizer)


--- Step 4: Loading and Preparing Data ---


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/220M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/221M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/458285 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/2698 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/2698 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Training set size: 2698
Validation set size: 300


In [17]:
train_ds

Dataset({
    features: ['dataset', 'id', 'messages', 'direction', 'metadata', 'text'],
    num_rows: 2698
})

In [18]:
train_ds["text"][0]

'<start_of_turn>user\nكمل هاد لقصة:\nال الجنين ويبقا مراقب النمو ديالو والصحة ديالواما هي ماكانتش كاتفلت الحصص ديالها فديك المؤسسة والا الوقت ديال سبور .. دارت نظام غذائي صحي ليها ... بعدات من القهوة بصفة نهائية .. ولات هي لي كاتبعد من الطالون بلا مايقول ليها لا حيديه لا لبسيهكانت ببساطة كادير اي حاجة عارفاها ماغاتضرش البيبي سواء كان حاضر ولا كان غايب وغاتبعد من أي حاجة غاتضر داك البيبي لي كان كايكبر فأحشائها ونهار على نهار كاتحس بحركاتو وكاتحس بيه معاهامن حسن حظها الحمالة ماجاتهاش بالوحم واخا هادشي كان جاها فشكل حيت أغلب لي كاتعرفهم ولدو كانو كايتشكاو من الوحم لدرجة دخلو لها الشك فراسها حتى سولات الطبيبة ولكن لي ماكانش عاجبها هو شكل كرشها لي تنفخ بحكم السبب الاول لي كان خلاها تولي تلعب الرياضة هو باش ماتبانش كرشها ولكن من سوء حظها هي ماكانتش شي وحدة لي رياضية من زماان باش تكون عندها شي نتيجة وااضحة ولكن ماكانتش كاتخبي كرشها على الموظفين والا على بوفسعلاقتها بداوود كانت فتقدم ملحوظ .. معتارفة لراسها بلي كاتبغيه ومتصالحة مع هاد الحقيقة ولي زاد صالحها معاها هي هضرتو .. فاش عرفات بلي دوك 